In [1]:
from keras.layers import Conv2D, Conv2DTranspose, Dense, Flatten, Reshape
from keras.models import Sequential, Model
from keras.utils.vis_utils import plot_model
from keras import losses
import numpy as np
import tensorflow as tf

In [2]:
x = np.load('../npy_files/features_All25.npy')
x.shape

(1878, 25, 5)

### Autoencoder Base

In [56]:
latent_dim = 4 
class Autoencoder_Base(Model):
  def __init__(self, latent_dim):
    super(Autoencoder_Base, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      Flatten(),
      Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      Dense(125, activation='sigmoid'),
      Reshape((25, 5))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder_Base(latent_dim)

In [57]:
autoencoder.compile(optimizer='adam', loss='mae')
autoencoder.build(input_shape=(1,25,5))

In [58]:
autoencoder.encoder.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_6 (Flatten)         (1, 125)                  0         
                                                                 
 dense_28 (Dense)            (1, 4)                    504       
                                                                 
Total params: 504
Trainable params: 504
Non-trainable params: 0
_________________________________________________________________


In [59]:
autoencoder.fit(x, x,
                epochs=5,
                shuffle=True)

Epoch 1/5
59/59 [==============================] - 0s 2ms/step - loss: 41.6211
Epoch 2/5
59/59 [==============================] - 0s 965us/step - loss: 41.4965
Epoch 3/5
59/59 [==============================] - 0s 969us/step - loss: 41.4430
Epoch 4/5
59/59 [==============================] - 0s 1ms/step - loss: 41.4271
Epoch 5/5
59/59 [==============================] - 0s 1ms/step - loss: 41.4222


### Autoencoder Dense

In [37]:
class Autoencoder_Dense(Model):
  def __init__(self, latent_dim):
    super(Autoencoder_Dense, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = Sequential([
      Flatten(),
      Dense(32, activation="relu",name='dense1'),
      Dense(16, activation="relu",name='dense2'),
      Dense(latent_dim, activation="relu",name='dense3')])

    self.decoder = Sequential([
      Dense(16, activation="relu",name='dedense1'),
      Dense(32, activation="relu",name='dedense2'),
      Dense(125, activation="sigmoid",name='dedense3'),
      Reshape((25, 5) , name='reshape1')
      ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder_Dense(8)

In [44]:
autoencoder.compile(optimizer='adam', loss='mae')
autoencoder.build(input_shape=(1,25,5))

In [49]:
autoencoder.encoder.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 125)               0         
                                                                 
 dense1 (Dense)              (None, 32)                4032      
                                                                 
 dense2 (Dense)              (None, 16)                528       
                                                                 
 dense3 (Dense)              (None, 8)                 136       
                                                                 
Total params: 4,696
Trainable params: 4,696
Non-trainable params: 0
_________________________________________________________________


In [50]:
autoencoder.decoder.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dedense1 (Dense)            (None, 16)                144       
                                                                 
 dedense2 (Dense)            (None, 32)                544       
                                                                 
 dedense3 (Dense)            (None, 125)               4125      
                                                                 
 reshape1 (Reshape)          (None, 25, 5)             0         
                                                                 
Total params: 4,813
Trainable params: 4,813
Non-trainable params: 0
_________________________________________________________________


In [51]:
autoencoder.fit(x, x,
                epochs=5,
                shuffle=True)

Epoch 1/5
59/59 [==============================] - 0s 1ms/step - loss: 41.2765
Epoch 2/5
59/59 [==============================] - 0s 1ms/step - loss: 41.2767
Epoch 3/5
59/59 [==============================] - 0s 1ms/step - loss: 41.2758
Epoch 4/5
59/59 [==============================] - 0s 1ms/step - loss: 41.2755
Epoch 5/5
59/59 [==============================] - 0s 2ms/step - loss: 41.2755


### Autoencoder conv2d

In [61]:
def CAE_mod(input_shape=(5,5,5), filters=[16, 32, 64, 3]):
    model = Sequential()
    if input_shape[0] % 8 == 0:
        pad3 = 'same'
    else:
        pad3 = 'valid'
    model.add(Conv2D(filters[0], 5, strides=2, padding='same', activation='relu', name='conv1', input_shape=input_shape))

    model.add(Conv2D(filters[1], 5, strides=2, padding='same', activation='relu', name='conv2'))

    model.add(Conv2D(filters[2], 2, strides=1, padding=pad3, activation='relu', name='conv3'))

    model.add(Flatten())
    model.add(Dense(units=filters[3], name='embedding'))
    model.add(Dense(units=filters[2]*int(input_shape[0]/8)*int(input_shape[0]/8), activation='relu'))

    model.add(Reshape((int(input_shape[0]/8), int(input_shape[0]/8), filters[2])))
    model.add(Conv2DTranspose(filters[1], 3, strides=2, padding=pad3, activation='relu', name='deconv3'))

    model.add(Conv2DTranspose(filters[0], 5, strides=2, padding='same', activation='relu', name='deconv2'))

    model.add(Conv2DTranspose(input_shape[2], 5, strides=2, padding='same', name='deconv1'))
    model.summary()
    return model


In [62]:
cae = CAE_mod()

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv2D)              (None, 3, 3, 16)          2016      
                                                                 
 conv2 (Conv2D)              (None, 2, 2, 32)          12832     
                                                                 
 conv3 (Conv2D)              (None, 1, 1, 64)          8256      
                                                                 
 flatten_7 (Flatten)         (None, 64)                0         
                                                                 
 embedding (Dense)           (None, 3)                 195       
                                                                 
 dense_30 (Dense)            (None, 0)                 0         
                                                                 
 reshape_10 (Reshape)        (None, 0, 0, 64)        

In [63]:
cae.compile(optimizer='adam', loss='mse')
cae.fit(x, x, batch_size=256, epochs=1)

ValueError: in user code:

    File "c:\Projects\TAU\DL-Workshop\Git Folder\Env\dl_work_mod\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "c:\Projects\TAU\DL-Workshop\Git Folder\Env\dl_work_mod\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Projects\TAU\DL-Workshop\Git Folder\Env\dl_work_mod\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "c:\Projects\TAU\DL-Workshop\Git Folder\Env\dl_work_mod\lib\site-packages\keras\engine\training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "c:\Projects\TAU\DL-Workshop\Git Folder\Env\dl_work_mod\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Projects\TAU\D

### Autoencoder LSTM

In [67]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

model = Sequential()
model.add(LSTM(4, activation='relu', input_shape=(25,5)))
model.add(RepeatVector(25))
model.add(LSTM(5, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(5)))
model.compile(optimizer='adam', loss='mse')

In [71]:
model.fit(x, x, epochs=200, verbose=1)

In [85]:
encoder = tfa.rnn.PeepholeLSTMCell(4 , activation=tf.nn.tanh,input_shape=(25,5),name="encoder")
rnn_enc = tf.keras.layers.RNN(encoder, return_sequences=True, return_state=True)

decoder = tfa.rnn.PeepholeLSTMCell(4 , activation=tf.nn.tanh,name="decoder")
rnn_dec = tf.keras.layers.RNN(decoder, return_sequences=True, return_state=True)

In [86]:
import tensorflow_addons as tfa
model = Sequential()
model.add(rnn_enc)
model.add(RepeatVector(25))
model.add(rnn_dec)
model.add(TimeDistributed(Dense(5)))
model.compile(optimizer='adam', loss='mse')

In [87]:
model.fit(x, x, epochs=10, verbose=1)

Epoch 1/10


ValueError: in user code:

    File "c:\Projects\TAU\DL-Workshop\Git Folder\Env\dl_work_mod\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "c:\Projects\TAU\DL-Workshop\Git Folder\Env\dl_work_mod\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Projects\TAU\DL-Workshop\Git Folder\Env\dl_work_mod\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "c:\Projects\TAU\DL-Workshop\Git Folder\Env\dl_work_mod\lib\site-packages\keras\engine\training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "c:\Projects\TAU\DL-Workshop\Git Folder\Env\dl_work_mod\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Projects\TAU\D

In [88]:
x.shape

(1878, 25, 5)

In [ ]:
x[:,:,]